# Regressão Linear com Bootstrap
Esse notebook tem como objetivo avaliar a relação entre a medida da cintura e do pescoço através de um modelo de regressão linear, com o propósito de demonstrar se a crença de que, colocando a calça ao redor do pescoço, ela serve.

In [ ]:
# Importando bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.utils import resample

# Reprodutibilidade
np.random.seed(123)

In [ ]:
# Dados amostrais
ds = pd.read_csv("data.csv")
pescoco = ds["Pescoço"]
cintura = ds["Cintura"]
dados = pd.DataFrame({'pescoco': pescoco, 'cintura': cintura})

In [ ]:
# Gráfico de dispersão
sns.scatterplot(x='pescoco', y='cintura', data=dados, color='blue')
modelo = LinearRegression()
modelo.fit(dados[['pescoco']], dados['cintura'])

# Previsões para a reta
x_vals = np.linspace(dados['pescoco'].min(), dados['pescoco'].max(), 100).reshape(-1, 1)
y_preds = modelo.predict(x_vals)
plt.plot(x_vals, y_preds, color='red', linewidth=2)
plt.title("Dispersão: Cintura vs Pescoço")
plt.xlabel("pescoco")
plt.ylabel("cintura")
plt.grid(True)
plt.show()

In [ ]:
# Coeficientes da regressão
print(f"Intercepto: {modelo.intercept_:.2f}, Inclinação (beta1): {modelo.coef_[0]:.2f}")

In [ ]:
# Bootstrap: 1000 reamostragens para IC
B = 5000
interceptos = np.zeros(B)
inclinacoes = np.zeros(B)

for i in range(B):
    amostra = resample(dados, replace=True)
    modelo_boot = LinearRegression().fit(amostra[['pescoco']], amostra['cintura'])
    interceptos[i] = modelo_boot.intercept_
    inclinacoes[i] = modelo_boot.coef_[0]

In [ ]:
# Intervalo de Confiança (percentis)
IC_intercepto = np.percentile(interceptos, [2.5, 97.5])
IC_inclinacao = np.percentile(inclinacoes, [2.5, 97.5])
print(f"IC Intercepto: {IC_intercepto}")
print(f"IC Inclinação (beta1): {IC_inclinacao}")

In [ ]:
# Histograma do bootstrap da inclinação (beta1)
plt.figure(figsize=(8, 5))
sns.histplot(inclinacoes, bins=20, kde=False, color="skyblue", edgecolor="black")
plt.axvline(IC_inclinacao[0], color='red', linestyle='--', label='IC 2.5%')
plt.axvline(IC_inclinacao[1], color='red', linestyle='--', label='IC 97.5%')
plt.title("Bootstrap de β₁")
plt.xlabel("β₁")
plt.ylabel("Frequência")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()